In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.signal
import numpy as np
import seaborn as sns

In [2]:
plt.rcParams["figure.figsize"]=[15,5]

In [3]:
raw_pp=pd.read_csv('pickandplace1_vibration.csv', usecols=['timestamp', 'data.ax', 'data.az'])
raw_pp.sort_values(by=['timestamp'], inplace=True)
raw_pp.reset_index(inplace=True)
raw_pp.dropna(axis=0, inplace=True)
raw_pp['timestamp'] =  pd.to_datetime(raw_pp['timestamp'])

In [4]:
#Load LD data and plot
raw_pp["acc"] = ( raw_pp["data.ax"]**2 + raw_pp["data.az"]**2) ** 0.5
acc=raw_pp[["timestamp", "acc"]]
test_pp=acc
# test_pp['acc_rolling_sum']=pd.Series.to_frame(test_pp.acc.rolling(100*10, center=True).sum())
test_pp['acc_rolling_std']=pd.Series.to_frame(test_pp.acc.rolling(100*2, center=True).std())


/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [5]:
%matplotlib qt

In [6]:
# plt.plot(test_pp['timestamp'], test_pp['acc'])
# plt.plot(test_pp['timestamp'], test_pp['acc_rolling_std'])

In [7]:
test_pp['detections']=pd.Series.to_frame(test_pp.acc_rolling_std.rolling(100, center=True).sum())

/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
plt.plot(test_pp['timestamp'], test_pp['acc']*100)
plt.plot(test_pp['timestamp'], test_pp['detections'])

In [9]:
plt.plot(test_pp['timestamp'], test_pp['detections'])

In [10]:
test_pp['boards']=0
test_pp['boards'][test_pp['detections']>=2]=1

/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
plt.plot(test_pp["timestamp"],test_pp["boards"])

In [12]:
working_times_raw=scipy.signal.find_peaks(test_pp.boards, distance=40*72, width=1)

In [13]:
working_times_raw_df=pd.DataFrame({"sample_number":working_times_raw[0], "working_time":working_times_raw[1]['widths']/72})

In [14]:
for x, row in working_times_raw_df.iterrows():
    working_times_raw_df.ix[x,'start_time']=test_pp.iloc[int(working_times_raw[1]['left_ips'][x])].timestamp
    working_times_raw_df.ix[x,'end_time']=test_pp.iloc[int(working_times_raw[1]['right_ips'][x])].timestamp
    working_times_raw_df.ix[x,'timestamp']=test_pp.iloc[int(working_times_raw[0][x])].timestamp

/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after remo

In [15]:
working_times_df=working_times_raw_df[working_times_raw_df.working_time>30]

In [16]:
working_times_df['event']=1

/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [17]:
raw_pp_current=pd.read_csv('out_T09:.csv', usecols=['timestamp', 'data.A1', 'data.A2', 'data.A3'])
raw_pp_current.sort_values(by=['timestamp'])
raw_pp_current.reset_index(inplace=True)
raw_pp_current['total_current']=raw_pp_current['data.A1']+raw_pp_current['data.A2']+raw_pp_current['data.A3']
raw_pp_current['timestamp']=pd.to_datetime(raw_pp_current['timestamp'])
raw_pp_current.index=raw_pp_current['timestamp']


FileNotFoundError: File b'out_T09:.csv' does not exist

In [ ]:
# working_times_df['energy']=raw_pp_current[(raw_pp_current['timestamp'] <= working_times_df['end_time']) & (raw_pp_current['timestamp'] >= working_times_df['start_time'])]['total_current'].sum()*230

In [ ]:
working_times_df['energy'] = working_times_df.apply(lambda x: raw_pp_current.loc[(raw_pp_current.timestamp <= x.end_time) & 
                                                            (x.start_time <= raw_pp_current.timestamp),
                                                            ['total_current']].sum()*230, axis=1)

In [ ]:
working_times_df.index=working_times_df.timestamp
working_times_df.drop('timestamp', axis=1, inplace=True)
working_times_df.drop('sample_number', axis=1, inplace=True)

In [ ]:
plt.plot(test_pp["timestamp"],test_pp["boards"])
plt.plot(raw_pp_current.timestamp, raw_pp_current['total_current'])

In [ ]:
PP_events=working_times_df[['event', 'working_time', 'energy']]

In [3]:
sns.distplot(PP_events.working_time)
plt.title('PP2 Working Times Histogram')

NameError: name 'PP_events' is not defined

In [ ]:
print('PP working time mode: ', PP_events.working_time.mode().mean())